# AutoEncoders

## Import Libraries

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Prepare Dataset

### Import the data from Git repo


In [0]:
movie_url = 'https://raw.githubusercontent.com/MarkIChen/deep-learning-unsupervised-learning/master/AutoEncoders/ml-1m/movies.dat'
user_url = 'https://raw.githubusercontent.com/MarkIChen/deep-learning-unsupervised-learning/master/AutoEncoders/ml-1m/users.dat'
rating_url = 'https://raw.githubusercontent.com/MarkIChen/deep-learning-unsupervised-learning/master/AutoEncoders/ml-1m/ratings.dat'

movies = pd.read_csv(movie_url, sep= '::', header = None, engine= 'python', encoding= 'latin-1')
users = pd.read_csv(user_url, sep= '::', header = None, engine= 'python', encoding= 'latin-1')
rating = pd.read_csv(rating_url, sep= '::', header= None, engine= 'python', encoding= 'latin-1')

### Preparing the traing set and the test set 

In [0]:
u1_train_url = 'https://raw.githubusercontent.com/MarkIChen/deep-learning-unsupervised-learning/master/AutoEncoders/ml-100k/u1.base'
u1_test_url = 'https://raw.githubusercontent.com/MarkIChen/deep-learning-unsupervised-learning/master/AutoEncoders/ml-100k/u1.test'

training_set = pd.read_csv(u1_train_url, delimiter= '\t')
test_set = pd.read_csv(u1_test_url, delimiter= '\t')

In [22]:
training_set.head

<bound method NDFrame.head of          1   1.1  5  874965758
0        1     2  3  876893171
1        1     3  4  878542960
2        1     4  3  876893119
3        1     5  3  889751712
4        1     7  4  875071561
...    ...   ... ..        ...
79994  943  1067  2  875501756
79995  943  1074  4  888640250
79996  943  1188  3  888640250
79997  943  1228  3  888640275
79998  943  1330  3  888692465

[79999 rows x 4 columns]>

In [23]:
test_set.head

<bound method NDFrame.head of          1     6  5  887431973
0        1    10  3  875693118
1        1    12  5  878542960
2        1    14  5  874965706
3        1    17  3  875073198
4        1    20  4  887431883
...    ...   ... ..        ...
19994  458   648  4  886395899
19995  458  1101  4  886397931
19996  459   934  3  879563639
19997  460    10  3  882912371
19998  462   682  5  886365231

[19999 rows x 4 columns]>

In [0]:
training_set = np.array(training_set, dtype= 'int')
test_set = np.array(test_set, dtype = 'int')

### Getting the number of users and movies




In [0]:
nb_users = int(max( max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max( max(training_set[:, 1]), max(test_set[:, 1])))

### Converting the data into an array with users in line and movies in columns(one hot encoding)

In [0]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1][data[:, 0] == id_users]
    id_ratings = data[:, 2][data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [0]:
training_set = convert(training_set)
test_set = convert(test_set)

In [38]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [39]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

### Converting the data into  Torch tensors

In [0]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [29]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

## Creating the architecture of the Network

In [0]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

In [0]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay = 0.5)

## Training the SAE

In [36]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    # increasing an additional dimension
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    # if the user did not rate any movie, we don't need to cal. it.
    if torch.sum(target.data > 0) > 0:
      output = sae(input) # which function?
      # To make sure that back-prob. will not update the target layer.
      target.require_grad = False
      # if the user did not rate the movie the prediction should be zero.
      output[target == 0] = 0
      loss = criterion(output, target)
      # We only concern about the none zero rating
      # Adding a small number is a math skill to avoid the sum is zero.
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      # deciding whether adding or decreasing the weights
      loss.backward()
      train_loss += np.sqrt(loss.data * mean_corrector)
      s += 1.
      # deciding the amount of change
      optimizer.step()
  print('epcoh '+str(epoch)+' loss: '+str(train_loss/s))

epcoh 1 loss: tensor(1.7623)
epcoh 2 loss: tensor(1.0964)
epcoh 3 loss: tensor(1.0533)
epcoh 4 loss: tensor(1.0385)
epcoh 5 loss: tensor(1.0307)
epcoh 6 loss: tensor(1.0267)
epcoh 7 loss: tensor(1.0238)
epcoh 8 loss: tensor(1.0220)
epcoh 9 loss: tensor(1.0206)
epcoh 10 loss: tensor(1.0199)
epcoh 11 loss: tensor(1.0191)
epcoh 12 loss: tensor(1.0184)
epcoh 13 loss: tensor(1.0179)
epcoh 14 loss: tensor(1.0175)
epcoh 15 loss: tensor(1.0174)
epcoh 16 loss: tensor(1.0170)
epcoh 17 loss: tensor(1.0167)
epcoh 18 loss: tensor(1.0165)
epcoh 19 loss: tensor(1.0164)
epcoh 20 loss: tensor(1.0163)
epcoh 21 loss: tensor(1.0161)
epcoh 22 loss: tensor(1.0160)
epcoh 23 loss: tensor(1.0161)
epcoh 24 loss: tensor(1.0159)
epcoh 25 loss: tensor(1.0158)
epcoh 26 loss: tensor(1.0156)
epcoh 27 loss: tensor(1.0155)
epcoh 28 loss: tensor(1.0152)
epcoh 29 loss: tensor(1.0128)
epcoh 30 loss: tensor(1.0113)
epcoh 31 loss: tensor(1.0103)
epcoh 32 loss: tensor(1.0082)
epcoh 33 loss: tensor(1.0075)
epcoh 34 loss: tens

## Testing the SAE

In [43]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0 :
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data * mean_corrector)
    s+= 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9511)
